### 1.0 Libraries and set-up

In [13]:
import ee
import os
import math
import geemap
import json
from shapely.geometry import shape
import geopandas as gpd

In [14]:
ee.Authenticate()

True

In [15]:
ee.Initialize()

In [16]:
region = 'YKD_weekly'
# region = 'AKCP_weekly'
# region = 'TUK_MKD_weekly'
# region = 'YKF_weekly'
folder = 'projects/alpod-412314/assets/' + region

In [17]:
image_list = ee.data.listImages(folder)
image_list = image_list.get('images')

In [31]:
rando_img = target_imgs[0]
print(rando_img.getInfo())
rando_properties = rando_img.getInfo()['properties']
print(rando_properties)

rando_img = rando_img.clip(combined_footprint)

export_task = ee.batch.Export.image.toDrive(
        image= rando_img,
        description='test_no_data',
        folder='',
        fileNamePrefix='test_no_data',
        region=combined_footprint,
        scale=10,
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    
export_task.start()

{'type': 'Image', 'bands': [{'id': 'water_occurance', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [22834, 11111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, -166.86880138983202, 0, -8.983152841195215e-05, 61.33436248535698]}], 'version': 1719797633687643, 'id': 'projects/alpod-412314/assets/YKD_weekly/03VVH_2016_25', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[-165.84319482922518, 61.334407594596996], [-166.09958530214465, 61.33440757070055], [-166.4841708075412, 61.33440755767273], [-166.86894410533003, 61.33440750232537], [-166.86893836721399, 60.33619919540345], [-166.676463585461, 60.33615307524081], [-166.42007326744408, 60.33615308977307], [-166.22778045805273, 60.33619929212763], [-166.03548764518138, 60.33615308084961], [-165.7790971444423, 60.336153061499246], [-165.52270678371238, 60.33615305980325], [-165.3304139971633, 60.33619932740822], [-165.1381211612177, 60.3361530645418

### 2.0 Images for target time period and region

In [19]:
target_years = list(range(2016, 2024))
target_years = [str(year) for year in target_years]
print(target_years)

# June
target_weeks = list(range(22, 27))
#August 
#target_weeks = list(range(31, 36))
# September
#target_weeks = list(range(35, 41))


target_weeks = [str(week) for week in target_weeks]
print(target_weeks)

max_times = len(target_years) * len(target_weeks)
print(max_times)

['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
['22', '23', '24', '25', '26']
40


In [20]:
target_imgs = []
target_polygons = []

for image in image_list:
    img_id = image['id']
    year = img_id.split('_')[-2]
    week = img_id.split('_')[-1]

    if week in target_weeks and year in target_years:
        img_ee = ee.Image(img_id)
        img_properties = img_ee.getInfo()['properties']
        img_coords = img_properties['system:footprint']['coordinates']
        polygon_ee = ee.Geometry.Polygon(img_coords)
        
        target_imgs.append(img_ee)
        target_polygons.append(polygon_ee)

print(len(target_imgs))

306


### 3.0 Export the ROI foot-print

In [21]:
p1 = target_polygons[0]
p2 = target_polygons[1]

combined_footprint = p1.union(p2)

for i in range(2, len(target_polygons)):
    p = target_polygons[i]
    combined_footprint = combined_footprint.union(p)

# combined_footprint_info = combined_footprint.getInfo()
# geojson_footprint = json.dumps(combined_footprint_info)
# geometry = shape(json.loads(geojson_footprint))
# roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{region}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{region}.shp')

### 4.0 Determine the observation counts for each pixel

In [22]:
obs_masks = []
for polygon in target_polygons:
    obs_mask = ee.Image.constant(1).clip(polygon)
    #obs_mask = obs_mask.set(1)
    obs_masks.append(obs_mask)
    
obs_collection = ee.ImageCollection(obs_masks)

pixel_observations = obs_collection.select('constant').sum()
#print(pixel_observations)


### 4.1 Visualize the observation counts

In [23]:
viz_obs = {
    'bands': ['constant'],
    'min': 0,
    'max': max_times,
    'palette': ['ff0000', 'ffff00', '00ff00']
}

Map = geemap.Map(center=(65, -135), zoom=4)
Map.addLayer(pixel_observations, viz_obs, f'{max_times}')
Map

Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### 5.0 Calculate the number of water occurances for each pixel

In [24]:
target_collection = ee.ImageCollection(target_imgs)
wtr_sum_img = target_collection.select('water_occurance').sum()
wtr_sum_img = wtr_sum_img.clip(combined_footprint)
print(wtr_sum_img.getInfo())

{'type': 'Image', 'bands': [{'id': 'water_occurance', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'dimensions': [12, 7], 'origin': [-168, 58], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-158.81741875650087, 61.23052765733927], [-158.8024522843577, 61.23051393955594], [-158.53850485582242, 61.23051386862255], [-158.27455731411695, 61.23051389030812], [-158.07659673534127, 61.23056200207664], [-157.87863604281475, 61.23051392429156], [-157.68067542567456, 61.230561973854776], [-157.4827148939237, 61.23051390657826], [-157.21876743052007, 61.230513916691315], [-157.02080673539527, 61.23056203920838], [-156.88864599989088, 61.230561926907775], [-156.88863985881972, 62.23182441090425], [-157.28475425913766, 62.231824559132846], [-157.68067542065853, 62.231824495749755], [-157.94462289368752, 62.23182455417869], [-158.20857045214677

### 6.0 Dived each pixels water occurance by total observations

In [25]:
wtr_proportion = wtr_sum_img.select('water_occurance').divide(pixel_observations.select('constant'))

In [26]:
print(wtr_proportion.getInfo())

{'type': 'Image', 'bands': [{'id': 'water_occurance', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [12, 7], 'origin': [-168, 58], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-158.81741875650087, 61.23052765733927], [-158.8024522843577, 61.23051393955594], [-158.53850485582242, 61.23051386862255], [-158.27455731411695, 61.23051389030812], [-158.07659673534127, 61.23056200207664], [-157.87863604281475, 61.23051392429156], [-157.68067542567456, 61.230561973854776], [-157.4827148939237, 61.23051390657826], [-157.21876743052007, 61.230513916691315], [-157.02080673539527, 61.23056203920838], [-156.88864599989088, 61.230561926907775], [-156.88863985881972, 62.23182441090425], [-157.28475425913766, 62.231824559132846], [-157.68067542065853, 62.231824495749755], [-157.94462289368752, 62.23182455417869], [-158.20857045214677, 62.23182451168546], [-158.7364654313869, 62.2318245697084

### 6.1 Visualize the water occurance proportions

In [29]:
viz_prop = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 1,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map = geemap.Map(center=(65, -135), zoom=4)
Map.addLayer(wtr_proportion, viz_prop, '.')
Map



Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### 7.0 Binary image based on number of occurances

In [12]:
time_fractions = [1/5, 1/6, 1/7, 1/10, 1/15]
#print(time_fractions)
water_thresholds = [time_fraction * max_times for time_fraction in time_fractions]
print(water_thresholds)

binary_images = {}
for threshold in water_thresholds:
    threshold_round = math.floor(threshold)
    wtr_img_binary = wtr_sum_img.gt(threshold)
    binary_images[threshold_round] = wtr_img_binary

print(binary_images)

[8.0, 6.666666666666666, 5.7142857142857135, 4.0, 2.6666666666666665]
{8: <ee.image.Image object at 0x165929460>, 6: <ee.image.Image object at 0x165a1fb90>, 5: <ee.image.Image object at 0x165a1d9a0>, 4: <ee.image.Image object at 0x165a1d8b0>, 2: <ee.image.Image object at 0x165a1ffb0>}


### 6.0 Visualize results

In [13]:
viz1 = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 1,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map = geemap.Map(center=(65, -135), zoom=4)

occurance_cnts = binary_images.keys()
print(occurance_cnts)

for cnt in occurance_cnts:
    Map.addLayer(binary_images[cnt], viz1, f'{region}, {cnt}, {target_weeks[0]}')


#Map.addLayer(combined_footprint, {}, 'roi_bounds')
Map


dict_keys([8, 6, 5, 4, 2])


Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [18]:
viz2 = {
    'bands': ['water_occurance'],
    'min': 0,
    'max': 4,
    'palette': ['ff0000', '00ff00', '0000ff']
}

Map2 = geemap.Map(center=(65,-135), zoom=4)
Map2.addLayer(wtr_sum_img , viz2, f'Occurence sums')
Map2

Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:


### 7.1 Export the .GeoTiff to Drive

In [13]:
for cnt in occurance_cnts:

    img = binary_images[cnt]

    export_task = ee.batch.Export.image.toDrive(
        image=img,
        description=f'{region}_count{cnt}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
        folder='ewebb_s2',
        fileNamePrefix=f'{region}_count{cnt}_years{target_years[0]}-{target_years[-1]}_weeks{target_weeks[0]}-{target_weeks[-1]}',
        region=combined_footprint,
        scale=10,
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    
    export_task.start()

### X.2 Export the .shp files to Drive??

### * Scrap workspace

In [ ]:
# binary_sum = weekly_sum.gt(3)

# vectors = binary_sum.reduceToVectors(
#     geometryType='polygon',
#     reducer=ee.Reducer.countEvery(),
#     scale=10,
#     maxPixels=1e13
# )
# # !!! Need to switch to a for-loop, becuase .getInfo() won't map to the server-side. 
# def get_geojson(vectors):
#     return ee.Feature(vectors).geometry().getInfo()

# vector_geoms = vectors.map(get_geojson)

# print(vector_geoms)

In [43]:
# np_img = geemap.ee_to_numpy(clipped_mosaic)

In [44]:
# t = target_collection.clip(combined_footprint)
# target_mosaic = target_collection.mosaic()
# clipped_mosaic = target_mosaic.clip(combined_footprint)

60
